In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [2]:
company_names = pd.read_excel("company_names.xlsx", header=None, names=["NameToFind", "del1", "del2", "FundooLinks", "Name", "Industry", "Address", "Ph No", "Website"])
company_names = company_names.drop(["del1", "del2"], axis=1)
company_names.head()

,NameToFind,FundooLinks,Name,Industry,Address,Ph No,Website
0,amazon,NaN,NaN,NaN,NaN,NaN,NaN
1,abengoa,NaN,NaN,NaN,NaN,NaN,NaN
2,abb,NaN,NaN,NaN,NaN,NaN,NaN
3,accenture,NaN,NaN,NaN,NaN,NaN,NaN
4,aditya birla,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
for ind in tqdm(company_names.index):
    search_link = "https://www.google.com/search?q="+company_names["NameToFind"][ind]+" fundoodata"
    source = requests.get(search_link).text
    soup = BeautifulSoup(source, 'html.parser')
    fundoo_link = soup.find_all('div', class_='kCrYT')
    if(fundoo_link):
        for link in fundoo_link[:3]:
            if(link.a):
                if("fundoodata" in link.a['href']):
                    company_link = link.a["href"].replace("/url?q=", "")
                    company_link = company_link[:(company_link.find(".html")+5)]
                    company_names["FundooLinks"][ind]= company_link
                    if ("fundoodata" in company_link):
                        details = requests.get(company_link).text
                        soup1 = BeautifulSoup(details, 'lxml')
                        if(soup1.find('div', class_="search-page-heading-red")):
                            company_names["Name"][ind] = " ".join(soup1.find('div', class_="search-page-heading-red").text.split())
                            if(soup1.find('div', class_="detail-line")):
                                temp3 = soup1.find('div', class_="detail-line")
                                if(temp3.a.text):
                                    company_names['Website'][ind] = temp3.a.text
                                hola = temp3.text.split()
                                if (temp3.a.text in hola):
                                    hola.remove(temp3.a.text)
                                phno = " ".join(hola)
                                company_names["Ph No"][ind] = phno
                                tempo = soup1.find('div', class_="search-page-right-pannel")
                                if(tempo is not None):
                                    a = tempo.text.split()
                                    try: 
                                        address = " ".join(a[(a.index('Address')+2):a.index(phno.split()[0])])
                                        company_names["Address"][ind] = address
                                    except IndexError:
                                        pass
                            industrysoup = soup1.find('div', class_="overview-box2")
                            if(industrysoup is not None):
                                industry = industrysoup.text.replace("\nIndustry\n", "")
                                company_names["Industry"][ind] = industry
                    break

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib

In [4]:
company_names.head()

,NameToFind,FundooLinks,Name,Industry,Address,Ph No,Website
0,amazon,https://www.fundoodata.com/companies-detail/Am...,Amazon Internet Services Pvt Ltd (Amazon Web S...,Information Technology (IT),"Brigade Gateway, World Trade Center, Bangalore...",(080) 41970000,aws.amazon.com
1,abengoa,NaN,NaN,NaN,NaN,NaN,NaN
2,abb,https://www.fundoodata.com/companies-detail/AB...,ABB India Ltd,"Electrical , Electronics","21st Floor,World Trade Centre, Dr Raj kumar Ro...",(080) 67143000,new.abb.com
3,accenture,https://www.fundoodata.com/companies-detail/Ac...,Accenture Services Pvt Ltd,Information Technology (IT),IBC Knowledge Park Corporation No 4/1 Bannergh...,(080) 41060000 41866600,www.accenture.com
4,aditya birla,https://www.fundoodata.com/companies-detail/Ad...,Aditya Birla Capital Ltd (Aditya Birla),BFSI,"First Floor, One Indiabulls Centre, Tower 1, 1...",(022) 43567000,adityabirlacapital.com


In [5]:
company_names.to_excel('output.xlsx')